In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-full-cold-manual/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

12 experiments


{'commit': 'c3bfe54b8a74a09839e433ab6bc386e37e4188a5',
 'id': '698469ea608e3359e23ca7295849e0aca721737e',
 'name': 'riant-duad',
 'params': {'dataset': {'path': 'bdsaglam/musique',
   'name': 'answerable',
   'split': 'validation'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.0,
   'system_prompt': 'cok/adapted.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'semi/cok.json',
   'n_shot': 3,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.5473727761688043,
  'f1': 0.6563944696497852,
  'fuzzy_match': 0.6032271410839884,
  '2hops': {'exact_match': 0.5559105431309904,
   'f1': 0.6624051882860094,
   'fuzzy_match': 0.6214057507987221},
  '3hops': {'exact_match': 0.5894736842105263,
   'f1': 0.7064689505111331,
   'fuzzy_match': 0.6473684210526316},
  '4hops': {'exact_match': 0.4419753086419753,
   'f1': 0.5438462593110768,
   'fuzzy_match': 0.4641975308641975},
  'gen_token_count': {'all': {'count': 2417.0,
    'mean': 82.89987587918908,
    'std':

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique")
    & (df["params.qa.few_shot_examples"].str.contains("semi")) 
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

df['params.qa.n_sc'] = df['params.qa.n_sc'].fillna(1)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

10 experiments after preprocessing


In [4]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [5]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
0,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,0.0,"(1,)"
1,ccot/format-thought.txt,cq.txt,semi/ccot.json,3,1,0.0,"(1,)"
2,cok/adapted.txt,cq.txt,semi/cok.json,0,1,0.0,"(1,)"
3,cok/adapted.txt,cq.txt,semi/cok.json,3,1,0.0,"(1,)"
4,cot/format-thought.txt,cq.txt,semi/cot.json,0,1,0.0,"(1,)"
5,cot/format-thought.txt,cq.txt,semi/cot.json,3,1,0.0,"(1,)"
6,cte/format-triplets-few.txt,cq.txt,semi/cte.json,0,1,0.0,"(1,)"
7,cte/format-triplets-few.txt,cq.txt,semi/cte.json,3,1,0.0,"(1,)"
8,direct/format-few.txt,cq.txt,semi/direct.json,0,1,0.0,"(1,)"
9,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.0,"(1,)"


In [6]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	cok/adapted.txt
	cot/format-thought.txt
	cte/format-triplets-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	semi/ccot.json
	semi/cok.json
	semi/cot.json
	semi/cte.json
	semi/direct.json
- params.qa.n_shot
	0
	3
- params.qa.n_sc
	1
- params.qa.temperature
	0.0
- params.run
	(1,)


In [7]:
df.to_json('exps.jsonl', orient='records', lines=True)